In [4]:
import os
import re
from pathlib import Path

def extract_metrics(eval_path):
    """Extracts precision, recall, and F1 from an evaluation file."""
    precision = recall = f1 = 0.0
    extracted = 0

    if not os.path.exists(eval_path):
        return (0.0, 0.0, 0.0, 0)

    with open(eval_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            if line.startswith("Precision:"):
                precision = float(line.split(":")[1])
            elif line.startswith("Recall:"):
                recall = float(line.split(":")[1])
            elif line.startswith("F1-score:"):
                f1 = float(line.split(":")[1])
            elif re.match(r'^https?://', line):
                extracted += 1

    return (precision, recall, f1, extracted)

def count_ground_truth(gt_path):
    """Counts non-empty lines (ground truth IRIs)."""
    if not os.path.exists(gt_path):
        return 0
    with open(gt_path, 'r', encoding='utf-8') as f:
        return sum(1 for line in f if line.strip())

def generate_latex_table(ontology_dir, eval_dir, gt_dir, requirements, caption, label):
    header = r"""\begin{table}[ht]
\centering
\caption{%s}
\label{tab:%s}
\resizebox{\textwidth}{!}{
\begin{tabular}{l""" % (caption, label)

    # Add 4 groups of 3 columns (P, R, F1) and 2 final columns
    header += "ccc" * len(requirements) + "rr" + "}\n"

    subheaders = ["\\textbf{Ontology}"]
    for req in requirements:
        subheaders += [f"\\multicolumn{{3}}{{c}}{{{req}}}"]
    subheaders += ["\\textbf{GT}", "\\textbf{Ext}"]

    metrics_headers = [""] + ["P & R & $F_1$"] * len(requirements) + ["", ""]
    
    table = header
    table += " & ".join(subheaders) + r" \\" + "\n"
    table += " & ".join(metrics_headers) + r" \\" + "\n"
    table += r"\midrule" + "\n"

    ontologies = sorted(os.listdir(ontology_dir))
    
    for ontology_file in ontologies:
        if not ontology_file.endswith(('.ttl', '.owl')):
            continue
        ontology_name = Path(ontology_file).stem
        row = [ontology_name]

        total_gt = 0
        total_extracted = 0

        for req_id in requirements:
            eval_file = os.path.join(eval_dir, ontology_name, f"{req_id}_evaluation.txt")
            gt_file = os.path.join(gt_dir, ontology_name, f"{req_id}.txt")

            P, R, F1, extracted = extract_metrics(eval_file)
            gt_count = count_ground_truth(gt_file)

            row += [f"{P:.2f}", f"{R:.2f}", f"{F1:.2f}"]
            total_gt += gt_count
            total_extracted += extracted

        row += [str(total_gt), str(total_extracted)]
        table += " & ".join(row) + r" \\" + "\n"

    table += r"\bottomrule" + "\n"
    table += r"\end{tabular}" + "\n"
    table += r"}" + "\n"
    table += r"\end{table}"

    return table


In [5]:
requirements = ["requirement1", "requirement2", "requirement3", "requirement4"]

latex_code = generate_latex_table(
    ontology_dir="Ontologies",
    eval_dir="ExtractedTerms",
    gt_dir="GroundtruthTerms",
    requirements=requirements,
    caption="Evaluation of ontology matching across requirement groups.",
    label="ontology-eval"
)

with open("Latex/ontology_evaluation_table.tex", "w", encoding="utf-8") as f:
    f.write(latex_code)


In [6]:
latex_code = generate_latex_table(
    ontology_dir="ODPs",
    eval_dir="ODPs/ExtractedTerms",
    gt_dir="ODPs/GroundtruthTerms",
    requirements=requirements,
    caption="Evaluation of ODPs matching across requirement groups.",
    label="ontology-eval"
)

with open("Latex/ODPs_evaluation_table.tex", "w", encoding="utf-8") as f:
    f.write(latex_code)